<br>

# Introdução



In [ ]:
import os
import pandas as pd
import geopandas as gpd

<br>

# Envelope

In [ ]:
# Get envelope
url = 'https://raw.githubusercontent.com/open-geodata/sp/main/data/shps/sp_250k_wgs84.geojson'
gdf_sp = gpd.read_file(url)

# Junta tudo
gdf_sp = gdf_sp.unary_union
gdf_sp = gdf_sp.buffer(1)
gdf_sp = gdf_sp.envelope

# Geodataframe
gdf_sp = gpd.GeoDataFrame(gpd.GeoSeries(gdf_sp))
gdf_sp = gdf_sp.rename(columns={0:'geometry'}).set_geometry('geometry')

<br>

# Lista *shapefiles*

In [ ]:
# Pasta dos Shapefiles
shp_dir = os.path.join('data', 'shp_files')

# Lista de shapefiles
list_shps_points = [i for i in os.listdir(shp_dir) if i.endswith('points.shp')]
list_shps_lines = [i for i in os.listdir(shp_dir) if i.endswith('lines.shp')]
list_shps_polygons = [i for i in os.listdir(shp_dir) if i.endswith('polygons.shp')]

<br>

## Pontos

In [ ]:
gdf_all = gpd.GeoDataFrame()
for shps_points in list_shps_points:
    print(shps_points)
    gdf = gpd.read_file(os.path.join(shp_dir, shps_points))

    # Data Level
    gdf = gdf[gdf['DataLevel'].isna()]
    gdf.drop('DataLevel', axis=1, inplace=True)    
    gdf_all = gdf_all.append(gdf)
    
# Results
gdf = gdf_all
display(gdf.head(5))
print(gdf.type)
gdf.plot()

In [ ]:
dict_lyr_points_translate = {
    # Limtes
    'Amusement Center': 'a',
    'Bank': 'Banco',
    'Beacon': 'a',
    'Boat Ramp': 'a',
    'Bridge': 'Ponte',
    'Building': 'Contrução',
    'Cemetery': 'Cemitério',
    'Church': 'Igreja',
    'City, 100K - 500K': 'Cidade Média',
    'City, 10K - 50K': 'Cidade pequena',
    'City, 1M - 10M': 'Cidade Grande',
    'City, > 10M': 'Metrópole',
    'City, Population Unknown': 'Cidade',
    'Country': 'País',
    'Dam': 'Barragem',
    'Danger': 'a',
    'Golf Course': 'Campo de GOlfe',
    'Hospital': 'Hospital',
    'Island': 'Ilha',
    'Landmark': 'a',
    'Library': 'a',
    'Lighthouse': 'a',
    'Lodging': 'a',
    'Military Installation': 'Área Militar',
    'Mine': 'Mina',
    'Picnic Area': 'Área de Picnic',
    'Post Office': 'Correio',
    'Restrooms': 'Hotel',
    'Rock': 'a',
    'School': 'Escola',
    'Shopping': 'Shopping',
    'Stadium': 'Estádio',
    'Summit': 'a',
    'Swimming Area': 'Área de Natação',
    'Tower': 'Torre',
    'Unknown Point Feature': '?',
    'Waterfall': 'Cachoeira',
    'Weigh Station': 'a',
}

for lyr in list(set(gdf['LAYER'])):
    if not lyr in dict_lyr_points_translate.keys():
        print("Traduzir layer: '{}'".format(lyr))
    else:
        print('Tudo Ok!')

In [ ]:
# Clip
gdf.reset_index(drop=True, inplace=True)
gdf = gpd.clip(gdf, gdf_sp)

# Replace
gdf.replace({'LAYER': dict_lyr_points_translate}, inplace=True)

# Replace
try: gdf['NAME'] = gdf['NAME'].apply(lambda x: x.replace('~[0x1e]', ' '))
except: pass

# Save File
gdf.to_file(os.path.join('data', 'tracksource.gpkg'), layer='pontos', driver='GPKG')

# Results
display(gdf.head(5))
print(gdf.type)
gdf.plot()

<br>

## Linhas

In [ ]:
gdf_all = gpd.GeoDataFrame()
for shps_lines in list_shps_lines:
    print(shps_lines)
    gdf = gpd.read_file(os.path.join(shp_dir, shps_lines))

    # Data Level
    gdf = gdf[gdf['DataLevel'].isna()]
    gdf.drop('DataLevel', axis=1, inplace=True)    
    gdf_all = gdf_all.append(gdf)
    
# Results
gdf = gdf_all
display(gdf.head(5))
print(gdf.type)
gdf.plot()

In [ ]:
dict_lyr_lines_translate = {
    # Limtes
    'International Political Boundary': 'Limite Administrativo Internacional',
    'Major Political Boundary (State)': 'Limite Estadual',
    
    # Cursos d'Água
    'River, Unknown Length': 'Rio',
    'Intermittent Stream or River': 'Rio Intermitente',
    'Stream': 'Córrego',
    
    # Outros
    'Trail': 'Trilha',
    'Airport Runway': 'Pista de Aeroporto',
    'Ferry Route': 'Ferry Boat',
    'Railroad, Class 1A': 'Ferrovia',
    'Pipeline': 'Tubulação',
    'Powerline': 'Linha de Transmissão',

    # Rodovias
    'Major/US Highway': 'Rodovia Expressa',
    'Interstate': 'Rodovia Interestadual',
    'Arterial Road': 'Rodovia Arterial',
    'Unpaved Road': 'Rodovia não Pavimentada',
    'Collector Road': 'Rodovia Coletora',
    'Residential Road': 'Rodovia Residencial',
    'Ramp': 'Rampa de Acesso',
}

for lyr in list(set(gdf['LAYER'])):    
    if not lyr in dict_lyr_lines_translate.keys():
        print("Traduzir layer: '{}'".format(lyr))
    else:
        print('Tudo Ok!')

In [ ]:
# Clip
gdf.reset_index(drop=True, inplace=True)
gdf = gpd.clip(gdf, gdf_sp)

# Replace
gdf.replace({'LAYER': dict_lyr_lines_translate}, inplace=True)

# Replace
gdf['NAME'] = gdf['NAME'].apply(lambda x: x.replace('~[0x1e]', ' '))

# Rename
dict_rename = {
    'RTBR-': 'BR-',
    'RTSP-': 'SP-',
}
gdf.replace(
    {'NAME': dict_rename},
    regex=True,
    inplace=True
)

# Save File
gdf.to_file(os.path.join('data', 'tracksource.gpkg'), layer='linhas', driver='GPKG')

# Results
display(gdf.head(5))
print(gdf.type)
gdf.plot()

<br>

## Poligonos

In [ ]:
gdf_all = gpd.GeoDataFrame()
for shps_polygons in list_shps_polygons:
    print(shps_polygons)
    gdf = gpd.read_file(os.path.join(shp_dir, shps_polygons))

    # Data Level
    gdf = gdf[gdf['DataLevel'].isna()]
    gdf.drop('DataLevel', axis=1, inplace=True)    
    gdf_all = gdf_all.append(gdf)
    
# Results
gdf = gdf_all
display(gdf.head(5))
print(gdf.type)
gdf.plot()

In [ ]:
dict_lyr_polygons_translate = {
    # Limtes
    'City Park': 'Parque',
    'Golf Course': '',
    'Intermittent Stream/River': 's',
    'Lake, 0.5 - 1 sq. mi.': 'Lago',
    'Major National Park': 'Parque Nacional',
    'Ocean': 'Oceano',
    'Orchard or Plantation': 's',
    'Reservation': 'Reservatório',
    'Salt': 's',
    'Scrub Area': 's',
    'Unknown Area Type': 's',
    'Wetland': 's',
    'Wooded Area': 's',
}

for lyr in list(set(gdf['LAYER'])):    
    if not lyr in dict_lyr_polygons_translate.keys():
        print("Traduzir layer: '{}'".format(lyr))
    else:
        print('Tudo Ok!')

In [ ]:
# Clip
gdf.reset_index(drop=True, inplace=True)
gdf['geometry'] = gdf.buffer(0)
gdf = gpd.clip(gdf, gdf_sp)

# Replace
gdf.replace({'LAYER': dict_lyr_polygons_translate}, inplace=True)

# Replace
try: gdf['NAME'] = gdf['NAME'].apply(lambda x: x.replace('~[0x1e]', ' '))
except: pass

# Rename
dict_rename = {
    'RTBR-': 'BR-',
    'RTSP-': 'SP-',
}
gdf.replace(
    {'NAME': dict_rename},
    regex=True,
    inplace=True
)

# Save File
gdf.to_file(os.path.join('data', 'tracksource.gpkg'), layer='poligonos', driver='GPKG')

# Results
display(gdf.head(5))
print(gdf.type)
gdf.plot()